In [173]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.metrics import mutual_info_score as MI
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import (
    SelectFromModel,
    SequentialFeatureSelector
)

In [195]:
def generate_data(
        n=1000, 
        n_rel=5, 
        n_irrel=30, 
        betas=[1, 2, 3, 4, 5, 6], 
        n_classes=10,
        dataset_variant=0,
    ):
    """Method generates synthetic data and target variable. 

    X1 are relevant features - first n_rel columns in X
    X2, X3, X4, and X5 are irrelevant features.

    Args:
        n: number of rows to generate
        n_rel: number of relevant features that will be generated from normal distribution
        n_iirel: number of irrelevant features that will be generated from normal distribution
        n_const: number of features with constant values
        betas: list of coefficients for calculating target variable (must be of len n_rel + 1)
            the first element is bias
        n_classes: number of classes in target variable, classes are distributed evenly
        dataset_variant: defines which type of irrelevant features will be included in dataset:
                        - 0: include irrelevant features (y is not created based on them),
                        - 1: include copy of relevant features with added gaussian noise,
                        - 2: include interactions between relevant variables.

    Returns:
        X: synthetic data
        y: target variable
    """
    assert len(betas) == (n_rel + 1), 'len of betas must be equal to (n_rel + 1)'

    # relevant features from normal distribution
    X1 = np.random.normal(0, 1, (n, n_rel))

    # target variable
    y = X1 @ np.array(betas[1:]).T + betas[0]
    y = pd.qcut(y, n_classes, labels=False) 

    if dataset_variant == 0:
        # irrelevant features from normal distribution
        X2 = np.random.normal(0, 1, (n, n_irrel))
        
        X = np.concatenate([X1, X2], axis=1)
    elif dataset_variant == 1:
        # relevant features with noise
        X3 = X1 + np.random.normal(0, 0.1, (n, n_rel))

        X = np.concatenate([X1, X3], axis=1)
    else:
        # second order interactions of relevant features
        X4 = np.empty((n,0), float)
        for i in range(n_rel - 1):
            for j in range(i + 1, n_rel):
                X4 = np.append(X4, np.expand_dims(X1[:, i] * X1[:, j], 1), axis=1)

        X = np.concatenate([X1, X4], axis=1)

    return X, y

In [198]:
def CMI(X, Y, Z):
  cmi = 0
  for z in np.unique(Z):
    cmi += MI(X[Z == z], Y[Z == z]) * (len(Z[Z == z]) / len(Z))
  return cmi

def interaction_gain(X1, X2, Y):
   joint_mi = MI(X2, Y) + CMI(X1, Y, X2)
   single_mis = MI(X1, Y) + MI(X2, Y)
   return joint_mi - single_mis

def CMIM(X, y):
  chosen_indices = []
  for iter_num in range(5):
    j_values = []
    for i in tqdm(range(X.shape[1]), f"Calculating iteration {iter_num+1}..."):
        if i in chosen_indices:
            j_values.append(-10000)
            continue
        J = MI(X[:, i], y)
        max_value = -10000
        for j in chosen_indices:
            curr_value = MI(X[:, i], X[:, j]) - CMI(X[:, i], X[:, j], y)
            if curr_value > max_value:
                max_value = curr_value
        j_values.append(J - max_value)
    chosen_indices.append(np.argmax(j_values))
  
  return chosen_indices

def JMIM(X, y):
  max_mi = -10000
  first_idx = None
  for i in range(X.shape[1]):
    curr_mi = MI(X[:, i], y)
    if curr_mi > max_mi:
        first_idx = i
        max_mi = curr_mi
  
  chosen_indices = [first_idx]
  for iter_num in range(4):
    j_values = []
    for i in tqdm(range(X.shape[1]), f"Calculating iteration {iter_num+1}..."):
      if i in chosen_indices:
          j_values.append(-10000)
          continue
      min_value = 10000
      for j in chosen_indices:
        curr_value = MI(X[:, j], y) + CMI(X[:, i], y, X[:, j])
        if curr_value < min_value:
            min_value = curr_value
      j_values.append(min_value)
    chosen_indices.append(np.argmax(j_values))
  
  return chosen_indices

def IGFS(X, y):
  chosen_indices = []
  for iter_num in range(5):
    j_values = []
    for i in tqdm(range(X.shape[1]), f"Calculating iteration {iter_num+1}..."):
      if i in chosen_indices:
          j_values.append(-10000)
          continue
      J = MI(X[:, i], y)
      inter_gain_sum = 0
      for j in chosen_indices:
        inter_gain_sum += interaction_gain(X[:, i], X[:, j], y)
      if len(chosen_indices) != 0:
        inter_gain_sum /= len(chosen_indices)
      j_values.append(J + inter_gain_sum)
    chosen_indices.append(np.argmax(j_values))
  
  return chosen_indices

def wrapper_criterion(X, y, criterion="bic"):
    k = X.shape[1]
    included = []
    best = None
    while True:
        value = []
        for i in range(k):
            if i not in included:
                model = sm.OLS(y, sm.add_constant(X[:, included + [i]])).fit()
                if criterion == "bic":
                  score_val = model.bic
                elif criterion == "aic":
                  score_val = model.aic
                value.append((score_val, i))
        if not value:
            break
        value.sort()
        new_score, new_feature = value[0]
        if best is None or new_score < best:
            included.append(new_feature)
            best = new_score
        else:
            break
    model = sm.OLS(y, sm.add_constant(X[:, included])).fit()
    return model, included

In [196]:
X, y = generate_data(
    betas=[6, 5, 4, 3, 2, 1],
    dataset_variant=0
)
X_discr = np.copy(X)
for i in range(X.shape[1]):
    X_discr[:, i] = pd.cut(X[:, i], bins=10, labels=False)

In [199]:
model, best_predictors = wrapper_criterion(X, y)
best_predictors

[0, 1, 2, 3, 4]

In [200]:
best_predictors = CMIM(X_discr, y)
best_predictors

Calculating iteration 5...: 100%|██████████| 35/35 [00:01<00:00, 23.77it/s]


[0, 1, 2, 3, 19]

In [201]:
best_predictors = JMIM(X_discr, y)
best_predictors

Calculating iteration 4...: 100%|██████████| 35/35 [00:01<00:00, 23.71it/s]


[0, 1, 2, 3, 20]

In [202]:
best_predictors = IGFS(X_discr, y)
best_predictors

Calculating iteration 5...: 100%|██████████| 35/35 [00:01<00:00, 19.29it/s]


[0, 1, 2, 3, 6]

In [203]:
sfs_model = LogisticRegression(max_iter=1000, penalty="l1", solver="liblinear").fit(X, y)
# sfs_model = RandomForestClassifier(max_depth=3).fit(X, y)
sfs_forward = SequentialFeatureSelector(
    sfs_model, n_features_to_select="auto", tol=0.001, direction="forward"
).fit(X, y)
np.argwhere(sfs_forward.get_support()*1 > 0).T[0]

array([0, 1, 2, 3, 4], dtype=int64)

In [228]:
# Divorce dataset, source https://www.kaggle.com/datasets/rabieelkharoua/split-or-stay-divorce-predictor-dataset
divorce = pd.read_csv("data/divorce.csv", sep=";")
X_divorce = divorce.drop("Class", axis=1).to_numpy()
y_divorce = divorce["Class"].to_numpy()

# AIDS classification dataset, source: https://www.kaggle.com/datasets/aadarshvelu/aids-virus-infection-prediction
aids = pd.read_csv("data/aids.csv")

# Heart attack prediction dataset, source: https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
heart = pd.read_csv("data/heart.csv")

# LOL Diamond FF15 dataset, source: https://www.kaggle.com/datasets/jakejoeanderson/league-of-legends-diamond-matches-ff15
lol = pd.read_csv("data/lol.csv")

# Water quality dataset, source: https://www.kaggle.com/datasets/adityakadiwal/water-potability
water = pd.read_csv("data/water.csv")

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994,0.05,7.78,0.00,1.95,0.040,0.10,0.03,0.03,1.37,0.00,...,0.197,14.29,1.00,0.005,3.57,2.13,0.09,0.06,0.03,1
7995,0.05,24.22,0.02,0.59,0.010,0.45,0.02,0.02,1.48,0.00,...,0.031,10.27,1.00,0.001,1.48,1.11,0.09,0.10,0.08,1
7996,0.09,6.85,0.00,0.61,0.030,0.05,0.05,0.02,0.91,0.00,...,0.182,15.92,1.00,0.000,1.35,4.84,0.00,0.04,0.05,1
7997,0.01,10,0.01,2.00,0.000,2.00,0.00,0.09,0.00,0.00,...,0.000,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,1
